# 1. IMPORTS

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim # Optimizers are algorithms that update nn weights 
# toolbox of "learning" algorithms for neural networks
# every time we train, (SGD, Adam, RMSProp) changes the weights so predictions get better
from torchvision import datasets, transforms
# datasets => gives us prebuilt dataset like (MNIST digits), CIFAT-10, ImageNet
# transforms => image preprocessing steps => convert to tensor/resize/normalize/rotate/flip
# We only use one transform => transform.toTensor() also scales pixel values from 0-255 => 0-1

from torch.utils.data import DataLoader
# DataLoader => takes dataset, cuts it into batches, shuffles it, feeds images to model
# Like a machine that hands images to the model 64 at a time.

print("Pytorch version:",torch.__version__)

Pytorch version: 2.9.1+cpu


# 2. TRANSFORMS
- Images are PIL Images (0-255). Neural networks need tensors (0-1).
- transform converts each Image into a 28x28 tensor.
- Transforms => image preprocessing step=> convert to tensor/resize,,,,etc
- Compose=> A list of steps to apply to each image/ It runs transforms in order
- combine multiple transforms into one. -  e.g=>transforms.Compose([transforms.Resize((28, 28)), transforms.ToTensor(), transforms.Normalize(0.5, 0.5)])

In [10]:
transform = transforms.Compose([
    transforms.ToTensor()
])
print("Transform:", transform)

Transform: Compose(
    ToTensor()
)


# 3. LOAD DATASET (MNIST)
MNIST = handwritten digits 0-9
train=True => training images (60,000)
train=False => test images (10,000)
MNIST images are stored in binary format:
train-images-idx3-ubyte
train-labels-idx1-ubyte....etc PyTorch reads them internally
batch=64 => Mean in each epoch they process 64 images so training happens like
1. take 64 images
2. compute predictions
3. compute loss
4. update weights
5. take next 64 images
6. repeat
   This happens until all 60,000 images are used once=>1 epoch
shuffle=True=> Shuffling=randomly mixing the data before each epoch
Why?
B/c if the model sees data in the same order everytime, it might learn patterns based on order, not content.
So we shuffle=randomize.
* Training loader => shuffle=True
* Test loader => shuffle=False(we don't mix test samples)

In [11]:
train_dataset = datasets.MNIST(root="./data", train=True, 
                              download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("Training samples:", len(train_dataset))
print("Testing samples:", len(test_dataset))

100.0%
100.0%
100.0%
100.0%

Training samples: 60000
Testing samples: 10000


# 4. SIMPLE NEURAL NETWORK
Beginner explanation:
- An image is 28x28 = 784 pixels.
- We flatten the image into a vector of size 784.
- Pass through 2 linear layers.
nn.Module => is the base class for ALL PyTorch neural networks.
We must inherit from it to create our own model
think: nn.Module = parent class
SimpleClassifier = our custom model
__init__ is the constructor.
runs once, when model is created.
self = "this object", just like this in Java or C#....
Inside the constructor we define:
* layers
* operations
* parameters
So self.flattern = nn.Flatten()
creates a flatten operation.

1- Flatten 28x28=>784
   self.flatten = nn.Flatten()
Images are 28x28.
Neural networks want a long vector: 784 numbers.
Flatten does that automatically.
Before feeding to the 1st fully connected (FC) layer, we flatten it:
 28×28  →  784 numbers in a 1D vector
So the FC layer must take 784 inputs, which is why:
 nn.Linear(784, 128)


2- First fully-connected layer
self.fc1 = nn.Linear(28*28, 128)
Each MNIST Image is 28x28 pixels, grayscale -> 1 channel.
Before feeding to the 1st fully connected (FC) layer, we flatten it:
28x28 -> 784 numbers in a 1D vector
So the FC layer must take 784 inputs, which is why:
nn.Linear(784, 128)
Input: 784
Output: 128
This is hidden layer 1.
What is 128 => It is a design choise (a hyperparameter).
 * Neural networks learn through many neurons.
 * More neurons =  can learn more complex patterns
 * Too many neurons = slower + risk of overfitting
 * Too few neurons = cannot learn enough patterns
For MNIST typical choices are:
Layer                               Common choices
first hidden layer                  64, 128, 256 neurons
So... reason for 128 is => it's simply good, commonly used number but also used 64, 256 for MNIST dataset.

Why use nn.Linear in a fully connected layer?
Fully connected (dense) layers work like:
 output[i] = w1*x1 + w2*x2 + ... + wn*xn + bias
This is a Linear Transformation -> exactly the job of nn.Linear.
👉 A fully connected layer connects every input neuron to every output neuron.
That's why we call them Linear layers.


3- Second fully-connected layer
self.fc2 = nn.Linear(128, 10)
128=>10
B/c MNIST has 10 classes(digits 0-9)

What is forward()?
The forward pass describes HOW data flows through network.
x = self.flatten(x) => image->flatten
x = torch.relu(self.fc1(x)) => pass through first layer -> apply activation (ReLU)
Without activation (only Linear):
Model is only able to learn a straight line relationship:
 y = Wx + b
This means the network CANNOT learn:
 * curves,
 * edges,
 * shapes,
 * non-linear patterns.
Digit recognition MUST detect curves -> so linear is not enough.
ReLU adds non-linearity:
 ReLU(x) = max(0, x)
It turns this: 
 📉 straight line model
 into
 📈 curve + shape learning model
This allows the network to learn complex digit shapes.
That's why:
 x = torch.relu(self.fc1(x))

x = self.fc2(x) => final output (logits for classes 0-9)
The forward method is like describing how electicity flows through a circuit.

Component	                 Why we use it
nn.Linear(784, 128)	         Flattened image has 784 pixels; 128 is chosen (not                                 fixed) to provide enough neurons for learning
ReLU activation	             Allows learning curved/complex shapes instead of just                              straight-line relationships
Fully connected (Linear)     Connects all inputs to all outputs → standard NN      layer                         building block                  
128 neurons	                 Good default value, chosen by humans, can be changed

In [13]:
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = fc2 = nn.Linear(128, 10) # 10 digits

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleClassifier()
print(model)

SimpleClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


# 5. LOSS & OPTIMIZER
- CrossEntropyLoss = best for classification
- Adam = modern optimizer
We are going to use CrossEntropyLoss instead of MSE (Mean Squared Error) b/c
 MSE
 * used for regression (predicting numbers), like:
 * house price
 * temperature
 * stock price
Not good for classification b/c it does not understand classes.

 CrossEntropyLoss
 * Used for classification, especially when:
 * Output = class scores
 * Labels = class index (0-9)
This loss automatically:
 * applies Softmax (convers scores -> probabilities)
 * compares predicted probability vs. correct label
 * gives higher loss when model is wrong
MSE = for numbers
CrossEntropy = for categories (like digits)


What is model.parameters()?
When we create a model:
    class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
Inside each nn.Linear, PyTorch creates:
 * weigths
 * biases
These are the learnable values simply means: 👉 “Give me all weights and biases in the model so I can update them.”

What is lr=0.001?
lr = learning rate
It controls how big the weight updates are.
Simple example:
 New weight = Old weight - lr * gradient
* lr = 1.0 -> huge steps (too fast, unstable)
* lr = 0.0000001 -> tiny steps (training very slow)
* lr = 0.001 -> very common good value
So:
👉 Learning rate decides how fast the model learns.

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 6. TRAINING LOOP
STEPS:
 1. Loop over batches
 2. Forward pass => model(x)
 3. Compute loss
 4. Zero gradients
 5. Backprop
 6. Update weights
What is an epoch?
One epoch = model sees ALL training images once
We decide the number:
 * Too small -> model doesn't learn enough
 * Too large -> model overfits / wastes time

Common values: 5, 10, 20.

What is model.train()?
PyTorch models have 2 modes:
.train() -> during training -> enables dropout, batchnorm updates
.eval() -> during testing -> disables dropout etc.
Even if model doesn't use dropout, it is good practice.

What is train_loader?
When we write at step 3:
 => train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
This:
  * takes all 60,000 MNIST images

  * groups them in batches of 64 images

  * randomizes order (because shuffle=True)

  * each loop gives:
      images = a batch of 64 images
      labels = 64 corresponding digit labels

So:
 for images, labels in train_loader:
means;
Each iteration gives 64 images + 64 correct answers.

Explanation of inside loop:
 1. outputs = model(images)
Feed one batch of images into the model.
THe model gives predicted class scores for each image.

2. loss = loss_fn(outputs, labels)

Compare predictions vs. true labels.

CrossEntropyLoss does:

Softmax to convert scores → probabilities

Computes loss based on how wrong predictions are

3. optimizer.zero_grad()

IMPORTANT STEP.

Gradients accumulate, so before computing new gradients:

👉 we reset old gradients to zero

✔ 4. loss.backward()

This:

Computes the gradient for EACH weight

Figure out which direction reduces loss

Think:
  Which changes will make the model better?

✔ 5. optimizer.step()

This is where LEARNING happens.

Using gradient info:
 new_weight = old_weight - lr * gradient
Adam does this in a smart way (better than simple SGD).

✔ 6. total_loss += loss.item()

We collect the loss of all batches to print avg loss per epoch.

🎯 SUPER SIMPLE SUMMARY
| Code                                             | Meaning                                  |
| ------------------------------------------------ | ---------------------------------------- |
| `loss_fn = CrossEntropyLoss()`                   | For classification problems              |
| `optimizer = Adam(model.parameters(), lr=0.001)` | Updates weights using gradients          |
| `model.train()`                                  | Sets model to training mode              |
| `train_loader`                                   | Gives batches of 64 images + labels      |
| `optimizer.zero_grad()`                          | Clears old gradients                     |
| `loss.backward()`                                | Computes new gradients                   |
| `optimizer.step()`                               | Updates all model weights                |
| `epoch`                                          | How many times model sees entire dataset |



Basic Concept:
📌 Step-by-step example

MNIST = 60,000 images
Batch size = 64
In one epoch:
  Batch 1 → images 0–63  
  Batch 2 → images 64–127  
  Batch 3 → images 128–191  
  ...
  Batch 937 → images 59936–60000
  
Then:
Epoch 2 starts

Again from image 0 to 60000
But this time the model is already better because weights were updated after each batch in epoch 1.
✔ Why do this multiple epochs?

Because:

👉 The more the model sees the data, the better the learning.

Training is repetition.

Batch_size = 64 
decide how many images model processes at one time.
Think of it like this:

📦 The full dataset = 60,000 images
📦 Batch size = 64

So in one epoch, the model process the dataset in small chunks:
 60000 images ÷ 64 per batch ≈ 937 batches
So in one epoch, the loop runs 937 times.
Each iteration:
 images → tensor of shape (64, 1, 28, 28)
 labels → tensor of shape (64)


In [18]:
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()  # reset
        loss.backward()        # compute gradients
        optimizer.step()       # update

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} => Loss: {total_loss:.4f}")

Epoch 1/15 => Loss: 27.6601
Epoch 2/15 => Loss: 23.2200
Epoch 3/15 => Loss: 19.9619
Epoch 4/15 => Loss: 15.3851
Epoch 5/15 => Loss: 14.0172
Epoch 6/15 => Loss: 11.9478
Epoch 7/15 => Loss: 9.2649
Epoch 8/15 => Loss: 7.9288
Epoch 9/15 => Loss: 8.1957
Epoch 10/15 => Loss: 6.6669
Epoch 11/15 => Loss: 4.3433
Epoch 12/15 => Loss: 5.4909
Epoch 13/15 => Loss: 3.6643
Epoch 14/15 => Loss: 5.2104
Epoch 15/15 => Loss: 2.0508


# 7. EVALUATION
During evaluation, we check how well our model performs on data it has never seen before (test set).
1st initialize counters: 
correct = 0, number of correctly predicted images
total = 0, total number of test images
We will use these to calculate accuracy.
2nd set model to evaluation mode:
 model.eval()
Why?
✔ Turns off dropout
✔ Turns off batch normalization updates

(You don’t need these during testing)

Training mode ≠ Evaluation mode

3rd: Disable gradient calculation:
 with torch.no_grad():
During testing → we do NOT update weights

So we don’t need gradients

This makes evaluation faster and uses less memory

4th Loop through the test data:
 for images, labels in test_loader:
images = batch of images from test set

labels = true answers (digits 0–9)

Remember:
test_loader yields batches (e.g., 64 images at a time)

5th. Pass the images through the model:
 outputs = model(images)
 
The model generates 10 numbers per image
These are the scores for digits 0–9
Example output for one image might look like:
 [-1.2, 0.3, 4.5, 2.1, -0.7, 1.0, 0.9, -2.0, 3.1, 0.5]
The largest score is the predicted digit.


6th. Find the predicted digit:
 predictions = torch.argmax(outputs, dim=1)
 
argmax returns the index of the highest number
dim=1 means "choose the max from the 10 output classes"

Example:
 outputs for 1 image: [-1, 0, 2, 0.5, -2, 1, 0.7, -1, 0.3, 0.2]
 argmax → 2
So prediction = digit 2.

7th. Count correct predictions:
 correct += (predictions == labels).sum().item()

(predictions == labels) → boolean tensor (True/False)
.sum() → counts True values
We add this to our correct counter

8th. Count total images:
 total += labels.size(0)
* labels.size(0) = batch size
   (e.g., 64 images)

9th. Calculate accuracy
   0.95 -> 95%

In [20]:
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)

Test Accuracy: 0.979


# 8. PREDICT A SINGLE IMAGE
  example_img, example_label = test_dataset[3]
  * Get the 4th image (index 3)
  * example_img -> image tensor (shape: [1, 28, 28])
  * example_label -> correct digit (0-9)

Add batch dimension:
  example_img.unsqueeze(0)
Why?
The model expects shape:
  (batch_size, channels, height, width)
Our image shape:
  (1, 28, 28)
After unsqueeze:
 (1, 1, 28, 28)
Now it looks like a batch of 1 image.

Pass through model:
 output = model(...)
Model gives 10 numbers (one for each digit).

Select best prediction
 pred = torch.argmax(output)


 .item() converts tensor -> Python number.

In [23]:
example_img, example_label = test_dataset[3]
model.eval()
with torch.no_grad():
    output = model(example_img.unsqueeze(0)) # add batch dimension
    pred = torch.argmax(output)

print("True Label:", example_label)
print("Predicted:", pred.item())

True Label: 0
Predicted: 0
